In [1]:
import pandas as pd
import numpy as np
import pickle
import json
import import_ipynb
import scipy
from scipy.optimize import minimize

from my_libs.calc_features import *
from sklearn.preprocessing import StandardScaler

importing Jupyter notebook from D:\Repo\PycharmProjects\gitPNTZ\PNTZ\my_libs\calc_features.ipynb


In [2]:
pd.options.display.max_columns = 1000

In [3]:
targets = ['Предел текучести','Врем. сопротивление']
dir_names = ['DATA/MODELS_RF/YS RF valid', 'DATA/MODELS_RF/H RF valid']

In [4]:
def load_model(dir_name, target):
    model = pickle.load(open(dir_name+'/RF_model_'+target+'.sav', 'rb'))
    ls_need_col = json.load(open(dir_name+'/ls_need_col', "r"))
#     try:
    scaler = StandardScaler()
    scale_data = json.load(open(dir_name+'/scaler', "r"))
    scaler.mean_ = scale_data[0]
    scaler.scale_ = scale_data[1]
#     except:
#         scaler=None
    return model, ls_need_col, scaler

In [5]:
X = pd.read_excel('DATA/modes_before_model/try.xlsx')

In [6]:
a = np.array([1,2,3,4,-12,9,-7])
a = np.abs(a)
a

array([ 1,  2,  3,  4, 12,  9,  7])

In [7]:
def model_pr(df):
    df = np.array(df).reshape(1, -1)
    return np.abs(model.predict(df) - 70)

In [11]:
ls_temp_z = [
    '1 зона по ВТР закалка', '2 зона по ВТР закалка',
    '3 зона по ВТР закалка'
]

# ls_temp_o = [
#     '1 зона ВТР и уставка отпуск',
#     '2 зона ВТР и уставка отпуск', '3 зона ВТР и уставка отпуск',
#     '4 зона ВТР и уставка отпуск', '5 зона ВТР и уставка отпуск'
# ]

ls_s_spr = ['Скорость прохождения трубы через спрейер, м/с']

ls_shag = [
    'шаг балок закалочная печь, сек', 'шаг балок отпускная печь, сек'
]

In [12]:
X_c_new = X.copy()
X_c_new[ls_temp_z] = X_c_new[ls_temp_z] + 20
X_c_new[ls_s_spr] = X_c_new[ls_s_spr] + 0.5
X_c = X.copy()
# X_c[ls_temp_o] = X[ls_temp_o] - 20
X_c[ls_shag] = X[ls_shag] - 6

In [13]:
X_c_new

,Врем. сопротивление,прогноз Врем. сопротивление,MAE Врем. сопротивление,MSE Врем. сопротивление,Предел текучести,прогноз Предел текучести,MAE Предел текучести,MSE Предел текучести,№ партии,марка стали,диаметр,толщина стенки,1 зона по ВТР закалка,2 зона по ВТР закалка,3 зона по ВТР закалка,"шаг балок закалочная печь, сек","Скорость прохождения трубы через спрейер, м/с",t˚ C трубы после спреера,1 зона ВТР и уставка отпуск,2 зона ВТР и уставка отпуск,3 зона ВТР и уставка отпуск,4 зона ВТР и уставка отпуск,5 зона ВТР и уставка отпуск,"шаг балок отпускная печь, сек",C,Mn,Si,P,S,Cr,Ni,Cu,Al,V,Ti,Nb,Mo,N,C-coef,Параметр закалка,Параметр отпуск,Параметр отпуск новый V,Величина зерна
0,77.5,105.275544,27.775544,771.480857,66,100.784531,34.784531,1209.963562,709046,37Г2Ф,127,12.5,850,850,860,36,0.72,95,685,685,673,673,656,36,0.36549,1.51234,0.28287,0.01129,0.00785,0.0951,0.10289,0.1397,0.0288,0.08827,0.00007,0.00011,0.00006,0.0077,0.670405,829.051634,18.748483,10.457088,12.520138


In [14]:
X_c

,Врем. сопротивление,прогноз Врем. сопротивление,MAE Врем. сопротивление,MSE Врем. сопротивление,Предел текучести,прогноз Предел текучести,MAE Предел текучести,MSE Предел текучести,№ партии,марка стали,диаметр,толщина стенки,1 зона по ВТР закалка,2 зона по ВТР закалка,3 зона по ВТР закалка,"шаг балок закалочная печь, сек","Скорость прохождения трубы через спрейер, м/с",t˚ C трубы после спреера,1 зона ВТР и уставка отпуск,2 зона ВТР и уставка отпуск,3 зона ВТР и уставка отпуск,4 зона ВТР и уставка отпуск,5 зона ВТР и уставка отпуск,"шаг балок отпускная печь, сек",C,Mn,Si,P,S,Cr,Ni,Cu,Al,V,Ti,Nb,Mo,N,C-coef,Параметр закалка,Параметр отпуск,Параметр отпуск новый V,Величина зерна
0,77.5,105.275544,27.775544,771.480857,66,100.784531,34.784531,1209.963562,709046,37Г2Ф,127,12.5,830,830,840,30,0.22,95,685,685,673,673,656,30,0.36549,1.51234,0.28287,0.01129,0.00785,0.0951,0.10289,0.1397,0.0288,0.08827,0.00007,0.00011,0.00006,0.0077,0.670405,829.051634,18.748483,10.457088,12.520138


In [ ]:
for target, dir_name in zip(targets,dir_names):
        model, ls_need_col, scaler = load_model(dir_name, target)
        X_a = X_c[ls_need_col].copy()
        X_b = X_c_new[ls_need_col].copy()
        bounds = [(i, j) for i, j in zip(X_a.iloc[0,:], X_b.iloc[0,:])]
        a = scipy.optimize.differential_evolution(model_pr, bounds)
        print('ВАПРОС a',model.predict(X_a))
        print('ВАПРОС b',model.predict(X_b))
        print(a)
        print('АТВЕТ ',model.predict(a['x'].reshape(1, -1)))